# Data Preparation

This notebook contains preprocessing steps that need to be run before the [Model Training](2-ModelTraining.ipynb) notebook.

The objective is to minimize the amount of data processing that the model trainer needs to do on loading each batch from disk, because we don't want to introduce an IO or CPU bottleneck that will slow down the whole training process.

So, the data will be written out to disk in a form that's as close as possible to the input format needed by the model.

Also, we'll store this data on an SSD volume because we need random access to it. Seek times on a spinning disk are a showstopper for this.

## Prerequisites

You first need to download the data files for the [Kaggle Instacart challenge](https://www.kaggle.com/c/instacart-market-basket-analysis) and unpack them into a directory called `../csv` (relative to this notebook).

After that we're going to read and parse the CSV files into Pandas dataframes, perform some joining and filtering, extract raw Numpy arrays and write them back out to disk into a directory called `h5`. The model trainer will then read them from there. Make sure that `h5` resides on an SSD (see above).

## RAM usage

Beware! This notebook requires around 7GB of RAM to run.

In [1]:
import numpy as np
import pandas as pd
import h5py
import shutil
import os
from itertools import *
from collections import defaultdict

csv_dir = '../csv/'
h5_dir = 'h5/'

path = os.path.join

## CSV parsing

First read the CSV files into Pandas. See the competition website for descriptions of these.

In [2]:
order_products_prior = pd.read_csv(path(csv_dir, 'order_products__prior.csv'), engine='c',
                                   dtype={'order_id':np.int32, 
                                          'product_id':np.int32, 
                                          'add_to_cart_order':np.int8, 
                                          'reordered':np.int8})

order_products_train = pd.read_csv(path(csv_dir, 'order_products__train.csv'), engine='c',
                                   dtype={'order_id':np.int32, 
                                          'product_id':np.int32, 
                                          'add_to_cart_order':np.int8, 
                                          'reordered':np.int8})

orders = pd.read_csv(path(csv_dir, 'orders.csv'), engine='c',
                     dtype={'order_id':np.int32,
                            'user_id':np.int32,
                            'order_number':np.int8,
                            'order_dow':np.int8,
                            'order_hour_of_day':np.int8
                           })

products = pd.read_csv(path(csv_dir, 'products.csv'), engine='c',
                       dtype={'product_id':np.int32,
                              'aisle_id':np.int8,
                              'department_id':np.int8
                             })

## Data preprocessing

Now we need to process the data to:

* Count the number of users and products, and the size of the biggest order
 * The model needs to know how big to make its inputs and embedding tables
* Filter out each user's first order, for now
 * None of the items in those are reorders (by definition) so they make things harder for the model
 * NB this means we can't make predictions based on just one order, which is a tradeoff worth revisiting later
* Retrieve pairs of consecutive order IDs for the same user
 * We're trying to predict the reorders in the second from the contents of the first

In [3]:
max_product_id = max(products.product_id)
max_product_id

49688

In [4]:
max_user_id = max(orders.user_id)
max_user_id

206209

`order_products` tells us what products were present in each order, and which of those were reorders. It also contains an `add_to_cart_order` column that we're ignoring for this project.

In [5]:
order_products = pd.concat([order_products_train, order_products_prior], axis=0
                          )[['order_id', 'product_id', 'reordered']]
order_products.head()

,order_id,product_id,reordered
0,1,49302,1
1,1,11109,1
2,1,10246,0
3,1,49683,0
4,1,43633,1


`return_orders` are all orders which were _not_ that user's first order. Then we join these onto `order_products`. The `orders` table also contains some other columns that we're not using, e.g. time-related data, so we'll drop these.

In [6]:
return_orders = orders.query('order_number > 1'
                            )[['order_id', 'user_id', 'order_number']]
return_orders.head()

,order_id,user_id,order_number
1,2398795,1,2
2,473747,1,3
3,2254736,1,4
4,431534,1,5
5,3367565,1,6


In [7]:
return_order_products = pd.merge(left=return_orders[['order_id']],
                                 right=order_products,
                                 on='order_id'
                                ).set_index('order_id')
return_order_products.head()

,product_id,reordered
order_id,,
2398795,196,1
2398795,10258,0
2398795,12427,1
2398795,13176,0
2398795,26088,1


Now we create two new data structures:

`order_sets` maps from order ID to the set of product IDs that were present in that order.

`reorder_sets` is the same, but only includes those which were reordered (i.e. the user had ordered them before at some point). This is indicated by a flag in `order_products` so we don't have to reconstruct the user's entire history to determine this, thankfully.

This will take a few minutes to run. But first, clear out some tables we don't need any more, to save RAM.

In [8]:
del order_products_prior
del order_products_train
del order_products
del orders
del products

In [9]:
# It might be much quicker to use numpy arrays here, and to rewrite
# the loop to group by the index values (i.e the order IDs) first

order_sets = defaultdict(set)
reorder_sets = defaultdict(set)

for row in return_order_products.itertuples():
  order_sets[row.Index].add(row.product_id)
  if row.reordered == 1:
    reorder_sets[row.Index].add(row.product_id)

(len(order_sets), len(reorder_sets))

(3139874, 2948968)

In [10]:
biggest_order_size = max(len(order) for order in order_sets.values())
biggest_order_size

145

Now we need to generate the order pairs for each user. This will also take a few minutes.

Some of the later orders will be in the Kaggle test set, which we're not using for this project (as our objective is slightly different from the Kaggle contest). So we filter down to only ones that we have data for.

In [11]:
# From the itertools docs -- get consecutive pairs from a sequence
def pairwise(iterable):
  "s -> (s0, s1), (s1, s2), (s2, s3), ..."
  a, b = tee(iterable)
  next(b, None)
  return izip(a, b)

def make_x_y(order_ids):
  pairs = list(pairwise(order_ids))
  return pd.DataFrame.from_records(pairs, columns=['x', 'y'])

valid = return_orders[return_orders.order_id.isin(return_order_products.index)]
sort_cols = ['user_id', 'order_number']
order_pairs = valid.sort_values(sort_cols).groupby(['user_id']).order_id.apply(make_x_y)
order_pairs.head()

x        y
user_id                    
1       0  2398795   473747
        1   473747  2254736
        2  2254736   431534
        3   431534  3367565
        4  3367565   550135

## Generate output data

Now we write the data out to disk in HDF5 format. This also takes quite some time.

The `order_sets` and `reorder_sets` are converted to zero-padded arrays, each as long as the biggest order, so we don't have to do that at training time. This is fine because there is no product ID 0. When training the model, Keras will filter out and ignore the zeros.

Do some more cleanup first though.

In [12]:
del return_orders
del return_order_products
del valid

In [13]:
u8 = np.dtype('uint8')
u16 = np.dtype('uint16')
u32 = np.dtype('uint32')
vu8 = h5py.special_dtype(vlen=u8)
vu16 = h5py.special_dtype(vlen=u16)

datapath = path(h5_dir, 'training_data.h5')

try:
  os.remove(datapath)
except Exception, e:
  print("Couldn't remove %s: %s" % (datapath, str(e)))

datafile = h5py.File(datapath)
num_rows = len(order_pairs)
orders_dataset = datafile.create_dataset('orders', (num_rows, biggest_order_size), dtype=u16)
reorders_dataset = datafile.create_dataset('reorders', (num_rows, biggest_order_size), dtype=u16)
users_dataset = datafile.create_dataset('users', (num_rows,), dtype=u32)
items_dataset = datafile.create_dataset('items', (num_rows,), dtype=vu16)
labels_dataset = datafile.create_dataset('labels', (num_rows,), dtype=vu8)

In [16]:
for i in xrange(num_rows):
  
  user_id = order_pairs.index[i][0]
  
  # The earlier of the two orders
  order1 = order_pairs.iloc[i, 0]

  # The later of the two orders, that we're trying to predict the contents of
  order2 = order_pairs.iloc[i, 1]
  
  # Create padded array from order1's order set
  order1_padded = np.zeros(biggest_order_size, dtype=np.uint16)
  order1_set = list(order_sets[order1])
  order1_padded[0:len(order1_set)] = order1_set
  
  # Create padded array from reorder set -- might not be one, some orders have no reorders
  reorder_padded = np.zeros_like(order1_padded)
  if order1 in reorder_sets:
    reorder_set = list(reorder_sets[order1])
    reorder_padded[0:len(reorder_set)] = reorder_set

  # Labels for each item in order1:
  # 1 if item is in next order, 0 otherwise
  order2_set = order_sets[order2]
  labels = np.array([1 if item in order2_set else 0 for item in order1_set])
  
  # Write data out to file
  orders_dataset[i] = order1_padded
  reorders_dataset[i] = reorder_padded
  users_dataset[i] = user_id
  items_dataset[i] = order1_set
  labels_dataset[i] = labels
  if i % 100000 == 0:
    print("Wrote row %d of %d" % (i, num_rows))

Wrote row 0 of 2933665
Wrote row 100000 of 2933665
Wrote row 200000 of 2933665
Wrote row 300000 of 2933665
Wrote row 400000 of 2933665
Wrote row 500000 of 2933665
Wrote row 600000 of 2933665
Wrote row 700000 of 2933665
Wrote row 800000 of 2933665
Wrote row 900000 of 2933665
Wrote row 1000000 of 2933665
Wrote row 1100000 of 2933665
Wrote row 1200000 of 2933665
Wrote row 1300000 of 2933665
Wrote row 1400000 of 2933665
Wrote row 1500000 of 2933665
Wrote row 1600000 of 2933665
Wrote row 1700000 of 2933665
Wrote row 1800000 of 2933665
Wrote row 1900000 of 2933665
Wrote row 2000000 of 2933665
Wrote row 2100000 of 2933665
Wrote row 2200000 of 2933665
Wrote row 2300000 of 2933665
Wrote row 2400000 of 2933665
Wrote row 2500000 of 2933665
Wrote row 2600000 of 2933665
Wrote row 2700000 of 2933665
Wrote row 2800000 of 2933665
Wrote row 2900000 of 2933665


Finally we can write out some scalar values that the model needs to know, and then close the file.

In [17]:
biggest_order_scalar = datafile.create_dataset('biggest_order_size', (1,), dtype=u16)
num_rows_scalar = datafile.create_dataset('num_rows', (1,), dtype=u32)
max_product_id_scalar = datafile.create_dataset('max_product_id', (1,), dtype=u16)
max_user_id_scalar = datafile.create_dataset('max_user_id', (1,), dtype=u32)
biggest_order_scalar[0] = biggest_order_size
num_rows_scalar[0] = num_rows
max_product_id_scalar[0] = max_product_id
max_user_id_scalar[0] = max_user_id
datafile.close()